In [1]:
# Import the python libraries required for the data anaysis visualizations as well as the API Key for Google Cloud
import matplotlib.pyplot as plt
import pandas as pd 
import scipy.stats as st 
import numpy as np 
from scipy.stats import linregress 
import json
from mapconfig import api_key
import requests
import pprint
import gmaps


# assign variables to city data csv filepath
city_data_loc="worldcity_data.csv"

# read the data from the csv into Pandas dataframe
city_data=pd.read_csv(city_data_loc)

# Output the dataframe to the console/terminal
city_data

,Unnamed: 0,City,Country,Code,Hemisphere,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
0,0,Adak,United States,US,Northern,50,-175,41.00,93,6.93,75
1,1,Honolulu,United States,US,Northern,20,-160,75.52,88,10.85,40
2,2,Barrow,United States,US,Northern,70,-155,24.80,62,16.11,90
3,3,Hilo,United States,US,Northern,20,-155,64.87,56,5.82,75
4,4,Anchorage,United States,US,Northern,60,-150,38.61,82,0.76,90
...,...,...,...,...,...,...,...,...,...,...,...
927,927,Hamilton,New Zealand,NZ,Southern,-40,175,69.84,77,8.05,75
928,928,Hastings,New Zealand,NZ,Southern,-40,175,53.53,71,1.99,60
929,929,Suva,Fiji,FJ,Southern,-20,180,75.20,83,10.29,75
930,930,Funafuti,Tuvalu,TV,Southern,-10,180,84.20,70,8.05,75


In [2]:
# create a variable which holds all of the latitude and longitude data fror the cities
coord=city_data[["Latitude", "Longitude"]]
# create a variable which holds all of the humidity data fro the cities
humid=city_data["Humidity"]

In [3]:
#plot a heatmap using Google Maps API to depict the relative humidities of all of the cities
gmaps.configure(api_key=api_key)
fig=gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(coord, weights=humid))
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Segregate the dataframe based on temperature range, humidity, cloudiness, and wind speeds to determine "ideal vacation cities" from city weather data
temprange_data=city_data[(city_data["Temperature"]<85) & (city_data["Temperature"]>69) & (city_data["Cloudiness"]<3) & (city_data["Wind_Speed"]<15) & (city_data["Humidity"]<50)]

# output the resulting reduced dataframe of vacation city options to the console/terminal
temprange_data

,Unnamed: 0,City,Country,Code,Hemisphere,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
28,28,Mexicali,Mexico,MX,Northern,35,-115,79.39,33,1.28,5
54,54,Torreón,Mexico,MX,Northern,25,-105,75.20,19,1.50,5
527,527,Baghdad,Iraq,IQ,Northern,35,45,80.60,22,2.24,0
578,578,Lahore,Pakistan,PK,Northern,30,75,82.40,48,2.93,0
582,582,Jaipur,India,IN,Northern,25,75,82.40,30,2.24,1
583,583,Ludhiana,India,IN,Northern,30,75,76.41,27,2.42,0
747,747,Ayacucho,Peru,PE,Southern,-15,-75,71.60,24,2.24,3
761,761,Santiago,Chile,CL,Southern,-35,-70,79.63,28,2.24,0
810,810,Poços de Caldas,Brazil,BR,Southern,-20,-45,82.29,38,1.92,5


In [5]:
# create an empty list to hold extracted hotels in vacation cities
lodging=[]

# create lists that hold the latitude and longitude data for the vaactiuon cities and assign to variables
lat_list=list(temprange_data["Latitude"])
lon_list=list(temprange_data["Longitude"])

# lopp that iterates through the vacation city data latitude and longitude lists and makes calls to Google Places API based on those coordinates 
for i in range(len(temprange_data)):

    # create variables that hold the current indexed value of the latitude and longitude 
    lat_indx=lat_list[i]
    lon_indx=lon_list[i]

    #create a vcarible that holds the API url to be called including the coordinate data the radius around the coordinates to search and the fact that lodging is the category of location to be searched for as well as the API key
    url=f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat_indx},{lon_indx}&radius=50000&type=lodging&key=" + api_key

    # make the API call and return the data in json format to a variable
    response=requests.get(url)
    poi_data=response.json()

    # append the retrieved hotel data for the locations to the lodging list
    lodging.append(poi_data["results"][0]["name"])

# output the lodging list to the console terminal 
print(lodging)

['Avi Resort & Casino', 'Motel Campo Real', 'گوندی قەواڵی', 'STARiHOTELS Bhatinda Punjab', 'Hotel Castle Bijaipur', 'STARiHOTELS Bhatinda Punjab', 'Casa Andina Standard Nasca', 'Hotel Hualum', 'Suntory Business Hotel']


In [6]:
# create a new dataframe that contains the segregated vacation city data and a column containing the hotel names
vacaloc_data=temprange_data.copy()
vacaloc_data["Hotel"]=lodging
vacaloc_data

,Unnamed: 0,City,Country,Code,Hemisphere,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel
28,28,Mexicali,Mexico,MX,Northern,35,-115,79.39,33,1.28,5,Avi Resort & Casino
54,54,Torreón,Mexico,MX,Northern,25,-105,75.20,19,1.50,5,Motel Campo Real
527,527,Baghdad,Iraq,IQ,Northern,35,45,80.60,22,2.24,0,گوندی قەواڵی
578,578,Lahore,Pakistan,PK,Northern,30,75,82.40,48,2.93,0,STARiHOTELS Bhatinda Punjab
582,582,Jaipur,India,IN,Northern,25,75,82.40,30,2.24,1,Hotel Castle Bijaipur
583,583,Ludhiana,India,IN,Northern,30,75,76.41,27,2.42,0,STARiHOTELS Bhatinda Punjab
747,747,Ayacucho,Peru,PE,Southern,-15,-75,71.60,24,2.24,3,Casa Andina Standard Nasca
761,761,Santiago,Chile,CL,Southern,-35,-70,79.63,28,2.24,0,Hotel Hualum
810,810,Poços de Caldas,Brazil,BR,Southern,-20,-45,82.29,38,1.92,5,Suntory Business Hotel


In [7]:

# repeat the code for the creation of variable holding the coorinates of the orginal city list and the creation of variable holding the humidity data from that dataframe for portability and readability
coord=city_data[["Latitude", "Longitude"]]
humid=city_data["Humidity"]

In [8]:
# replot the heat map of humidity for the orginal ~1000 city list of world cities
fig=gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(coord, weights=humid))

In [9]:
# create variable containing the coordinates for the cities in the vacation city data
markloc_data=temprange_data[["Latitude", "Longitude"]]

# create the list of labels for the markers for the hotels 
labels=["Avi Resort and Casino in Mexicali Mexico", "Motel Campo Real in Torreon Mexico", "گوندی قەواڵی in Baghdad Iraq", "STARiHOTELS Bhatinda Punjab in Lahore Pakistan", "Hotel Castle Bijaipur in Jaipur India", "STARiHOTELS Bhatinda Punjab in Ludhiana India" , "Casa Andina Standard Nasca in Ayacucho Peru", "Hotel Hualum in Santiago Chile", "Suntory Business Hotel in Pocos de Caldas Brazil"]

In [10]:
# plot the markers and the pop up labels for the hotels on the world humidity heatmap 
markers=gmaps.marker_layer(markloc_data, info_box_content=labels)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))